In [1]:
# Uses the current state of the alkane model to construct an ASE atoms object
# for visualisation purposes.
def mk_ase_config(ibox, Nbeads, Nchains):
    
    # Create and populate ASE object
    model_positions = np.empty([Nchains*Nbeads, 3])
    cell_vectors = mdl.box_get_cell(ibox)
    displace = 0.5*(cell_vectors[0]+cell_vectors[1]+cell_vectors[2])
    
    for ichain in range(0, Nchains):
        model_positions[Nbeads*ichain:Nbeads*ichain+Nbeads] = mdl.alkane_get_chain(ichain+1, ibox)

    for iatom in range(0,Nbeads*Nchains):
        model_positions[iatom] = model_positions[iatom] + displace
    
    confstring = "C"+str(Nbeads*Nchains)
    
    box_config = Atoms(confstring, positions=model_positions*(1.5/0.4), pbc=True, cell=cell_vectors*(1.5/0.4)) 

    return box_config
    
# Takes an ASE atoms object or list thereof and creates a customised ngl viewer
# with appropriate settings for our bulk alkane chain models
def vis_chains(vis_config):
    
    met = 0.35
    rad = 0.8
    
    colours = ['#FF1111','#FFAAAA', '#DDDDDD', '#1111FF', '#AAAAFF']
    ncols = len(colours)
    
    sel=list()
    for icol in range(ncols):
        sel.append(list())
    
    # Create lists for each colour
    for ichain in range(0, Nchains):
        icol = ichain%ncols
        for ibead in range(Nbeads):
            iatom = ichain*Nbeads + ibead
            sel[icol].append(iatom)
            
    v = view(vis_config, viewer='ngl')                   
    v.view.clear_representations()
    v.view.add_representation('unitcell', color='#000000')
    
    for icol in range(ncols):
        v.view.add_representation('ball+stick', selection=sel[icol], color=colours[icol], radius=rad, metalness=met)

    return v

In [2]:
import numpy as np  # Numpy
import math as m    # Maths

import hs_alkane.alkane as mdl   # Fortran library we'll used to model the alkane chain

In [3]:
Nbeads  = 4    # Butane
Nchains = 250  # 250 chains in chain.xmol

# Initialise the simulation box and alkane module ready to hold chains
mdl.box_initialise()             
mdl.alkane_set_nbeads(Nbeads)    
mdl.alkane_set_nchains(Nchains)  
mdl.alkane_initialise()          

# Read from chain.xmol
mdl.io_read_xmol()               # Read from chain.xmol

In [4]:
from ase import Atoms
from ase.visualize import view

ase_config = mk_ase_config(1, Nbeads, Nchains)
vis_chains(ase_config)

### Sanity check initial configuration

In [5]:
# Loop over chains and sanity check configuration
ibox = 1
for ichain in range(0, Nchains):
    
    # Checks if internal geometry of chain (bonds/angles) are consistent with the model
    geom_flag = mdl.alkane_check_chain_geometry(ichain+1, ibox)
    if geom_flag != 0:
        print("Incorrect geometry for chain ",ichain)
       
    
# Checks if beads on any two chains overlap
overlap_flag = mdl.alkane_check_chain_overlap(ibox)
if overlap_flag != 0:
    print("Overlaps between chains in configuration")
else:
    print("No overlaps between chains found in configuration")
    

No overlaps between chains found in configuration


### Adjust density of initial configuration and visualise

In [6]:
# Scale the simulation box isotropically by 110%. Chain positions (first bead) scale with the box
mdl.alkane_box_scale(ibox, 1.1, 1.1, 1.1)

ase_config = mk_ase_config(1, Nbeads, Nchains)
vis_chains(ase_config)

In [7]:
# Put this back how it was
mdl.alkane_box_scale(ibox, 1.0/1.1, 1.0/1.1, 1.0/1.1)

In [8]:
# Try compressing too far
mdl.alkane_box_scale(ibox, 0.8, 0.8, 0.8)

# Checks if beads on any two chains overlap
overlap_flag = mdl.alkane_check_chain_overlap(ibox)
if overlap_flag != 0:
    print("Overlaps between chains in configuration")
else:
    print("No overlaps between chains found in configuration")

ase_config = mk_ase_config(1, Nbeads, Nchains)
vis_chains(ase_config)

Overlaps between chains in configuration


In [9]:
# Put this back how it was
mdl.alkane_box_scale(ibox, 1.0/0.8, 1.0/0.8, 1.0/0.8)

In [10]:
mdl.box_get_cell(ibox)

array([[  7.120546,   0.      ,   0.      ],
       [  1.584668,   7.85993 ,   0.      ],
       [ -1.815922,   2.549568, -10.042803]])

In [11]:
delta_cell = np.array([[5.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

In [12]:
mdl.alkane_change_box(ibox,delta_cell)

In [13]:
mdl.box_get_cell(ibox)

array([[ 12.120546,   0.      ,   0.      ],
       [  1.584668,   7.85993 ,   0.      ],
       [ -1.815922,   2.549568, -10.042803]])

In [14]:
ase_config = mk_ase_config(1, Nbeads, Nchains)
vis_chains(ase_config)

In [15]:
mdl.alkane_change_box(ibox,-delta_cell)

## Monte Carlo loop

In [23]:
# Reset
mdl.box_initialise()       
mdl.alkane_set_nbeads(Nbeads)    
mdl.alkane_set_nchains(Nchains)        
mdl.alkane_initialise()          

# Read from chain.xmol
mdl.io_read_xmol()               # Read from chain.xmol

In [24]:
mdl.alkane_get_dr_max()
mdl.alkane_set_dr_max(0.0001)

In [25]:
ase_config = mk_ase_config(1, Nbeads, Nchains)
vis_chains(ase_config)

In [26]:
Nsweeps = 100
isweep = 0

num_acc_trans = 0
num_att_trans = 0

while isweep < Nsweeps:

    # One move per degree of freedom at each sweep. Here we only
    # have translations so Nchains moves per sweep
    imove = 0
    while imove < Nchains:

        # Pick a random chain numbered from 0 to Nchains
        ichain = np.random.randint(0, high=Nchains)

        # Backup old chain positions
        current_chain = mdl.alkane_get_chain(ichain+1, ibox)
        backup_chain = current_chain.copy()

        # Attempt a Monte Carlo move
        num_att_trans += 1
        boltz = mdl.alkane_translate_chain(ichain+1,ibox)
        
        # Reject according to Metropolis criterion
        if (np.random.random() < boltz):
            # Move accepted
            num_acc_trans += 1
        else:
            # Reject move
            #current_chain = [backup_chain[ibead] for ibead in range(Nbeads)]
            for ibead in range(Nbeads):
                current_chain[ibead] = backup_chain[ibead]
            

        imove += 1
            
    isweep += 1
    
frac_acc_trans = num_acc_trans/num_att_trans
print("Accepted ",100*frac_acc_trans,"% of attempted chain translation moves")
    

Accepted  46.876 % of attempted chain translation moves


In [27]:
ase_config = mk_ase_config(1, Nbeads, Nchains)
vis_chains(ase_config)

In [22]:
# Checks if beads on any two chains overlap
overlap_flag = mdl.alkane_check_chain_overlap(ibox)
if overlap_flag != 0:
    print("Overlaps between chains in configuration")
else:
    print("No overlaps between chains found in configuration")

No overlaps between chains found in configuration
